### PDF Viewer Mobile Application Code below

In [4]:
import tkinter as tk
from tkinter import filedialog, messagebox
from PIL import Image, ImageTk
import fitz  # PyMuPDF


class PDFViewer:
    def __init__(self, master, pdf_path=None):
        self.master = master
        self.master.title("PDF Viewer")
        self.pdf_path = pdf_path
        self.page_index = 0
        self.zoom_factor = 1.0
        self.rotation = 0

        # Canvas for image
        self.canvas = tk.Label(master)
        self.canvas.pack(expand=True)

        # Control panel
        self.nav_frame = tk.Frame(master)
        self.nav_frame.pack(pady=10)

        # Buttons
        self.prev_btn = tk.Button(self.nav_frame, text="⏮️ Previous", command=self.prev_page)
        self.prev_btn.grid(row=0, column=0)

        self.next_btn = tk.Button(self.nav_frame, text="Next ⏭️", command=self.next_page)
        self.next_btn.grid(row=0, column=1)

        self.zoom_in_btn = tk.Button(self.nav_frame, text="🔍 Zoom In", command=self.zoom_in)
        self.zoom_in_btn.grid(row=0, column=2)

        self.zoom_out_btn = tk.Button(self.nav_frame, text="🔎 Zoom Out", command=self.zoom_out)
        self.zoom_out_btn.grid(row=0, column=3)

        self.rotate_btn = tk.Button(self.nav_frame, text="🔄 Rotate", command=self.rotate_page)
        self.rotate_btn.grid(row=0, column=4)

        self.page_label = tk.Label(self.nav_frame, text="Page 0")
        self.page_label.grid(row=0, column=5)

        self.open_btn = tk.Button(self.nav_frame, text="📂 Open PDF", command=self.open_pdf_dialog)
        self.open_btn.grid(row=0, column=6)

        if pdf_path:
            self.load_pdf(pdf_path)

    def open_pdf_dialog(self):
        file_path = filedialog.askopenfilename(filetypes=[("PDF files", "*.pdf")])
        if file_path:
            self.load_pdf(file_path)

    def load_pdf(self, path):
        try:
            self.doc = fitz.open(path)
            self.total_pages = len(self.doc)
            self.page_index = 0
            self.zoom_factor = 1.0
            self.rotation = 0
            self.show_page()
        except Exception as e:
            messagebox.showerror("Error", f"Failed to load PDF:\n{e}")

    def render_page(self, index):
        page = self.doc.load_page(index)
        mat = fitz.Matrix(self.zoom_factor, self.zoom_factor).preRotate(self.rotation)
        pix = page.get_pixmap(matrix=mat)
        img = Image.frombytes("RGB", [pix.width, pix.height], pix.samples)
        return ImageTk.PhotoImage(img)

    def show_page(self):
        img = self.render_page(self.page_index)
        self.canvas.configure(image=img)
        self.canvas.image = img
        self.page_label.config(text=f"Page {self.page_index + 1} / {self.total_pages}")

    def next_page(self):
        if self.page_index < self.total_pages - 1:
            self.page_index += 1
            self.show_page()

    def prev_page(self):
        if self.page_index > 0:
            self.page_index -= 1
            self.show_page()

    def zoom_in(self):
        self.zoom_factor += 0.2
        self.show_page()

    def zoom_out(self):
        self.zoom_factor = max(0.4, self.zoom_factor - 0.2)
        self.show_page()

    def rotate_page(self):
        self.rotation = (self.rotation + 90) % 360
        self.show_page()


### Gallery Mobile Application Code below

In [5]:
import os
from PIL import Image, ImageTk


class Gallery:
    def __init__(self, master, folder_path=None):
        self.master = master
        self.master.title("Gallery Viewer")
        self.folder_path = folder_path
        self.index = 0
        self.image_list = []
        self.rotation = 0

        self.image_label = tk.Label(master)
        self.image_label.pack(expand=True)

        self.control_frame = tk.Frame(master)
        self.control_frame.pack(pady=10)

        self.prev_btn = tk.Button(self.control_frame, text="⏮️ Previous", command=self.prev_image)
        self.prev_btn.grid(row=0, column=0)

        self.next_btn = tk.Button(self.control_frame, text="Next ⏭️", command=self.next_image)
        self.next_btn.grid(row=0, column=1)

        self.rotate_btn = tk.Button(self.control_frame, text="🔄 Rotate", command=self.rotate_image)
        self.rotate_btn.grid(row=0, column=2)

        self.open_btn = tk.Button(self.control_frame, text="📂 Open Folder", command=self.open_folder)
        self.open_btn.grid(row=0, column=3)

        self.name_label = tk.Label(self.control_frame, text="")
        self.name_label.grid(row=0, column=4)

        if folder_path:
            self.load_images(folder_path)

    def open_folder(self):
        path = filedialog.askdirectory()
        if path:
            self.load_images(path)

    def load_images(self, path):
        supported = ('.png', '.jpg', '.jpeg', '.gif', '.bmp', '.webp')
        self.image_list = [os.path.join(path, f) for f in os.listdir(path) if f.lower().endswith(supported)]
        self.index = 0
        self.rotation = 0
        if not self.image_list:
            messagebox.showinfo("No Images", "No supported image files found in the selected folder.")
        else:
            self.show_image()

    def render_image(self, img_path):
        img = Image.open(img_path)
        img = img.rotate(self.rotation, expand=True)
        img.thumbnail((800, 600))
        return ImageTk.PhotoImage(img)

    def show_image(self):
        if not self.image_list:
            return
        img = self.render_image(self.image_list[self.index])
        self.image_label.configure(image=img)
        self.image_label.image = img
        self.name_label.config(text=f"{os.path.basename(self.image_list[self.index])} ({self.index + 1}/{len(self.image_list)})")

    def next_image(self):
        if self.index < len(self.image_list) - 1:
            self.index += 1
            self.rotation = 0
            self.show_image()

    def prev_image(self):
        if self.index > 0:
            self.index -= 1
            self.rotation = 0
            self.show_image()

    def rotate_image(self):
        self.rotation = (self.rotation + 90) % 360
        self.show_image()


### Sample Try & Run Output

In [6]:
if __name__ == "__main__":
    root = tk.Tk()
    root.geometry("850x650")

    # To launch PDF Viewer:
    # viewer = PDFViewer(root, "your_pdf_path.pdf")

    # To launch Gallery Viewer:
    # gallery = Gallery(root, "your_images_folder")

    root.mainloop()
